In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import clean_up as cln 

In [2]:
# import "Annual_Parking_Study_Data" to creat 'pkdt' dataframe
# read data
col_names = ['Date Time','Unitdesc','Side', 'Parking_Spaces', 'Total_Vehicle_Count', 'Construction', 'Event Closure']
pkdt = cln.read_data(r'.\raw_data\Annual_Parking_Study_Data.csv', col_names)
pkdt = pkdt.dropna()

C:\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [3]:
#filter the data with 'Construction' and 'Event' column are "No"
values = ['Yes', 'yes', 'ERROR: #N/A']
pkdt = cln.data_filter(pkdt, 'Construction', values)
pkdt = cln.data_filter(pkdt, 'Event Closure', ['Yes'])
pkdt = cln.data_filter(pkdt, 'Parking_Spaces',[0])

pkdt = pkdt.drop('Construction', axis = 1)
pkdt = pkdt.drop('Event Closure', axis = 1)

In [4]:
#group rows with same 'Time' and 'Unitdesc', for other columns, find the sum
pkdt = pkdt.groupby(['Date Time', 'Unitdesc'], as_index = False)['Parking_Spaces', 'Total_Vehicle_Count'].sum()

In [5]:
#find the occupancy and append it to the pkdt
Occupancy = np.minimum(pkdt['Total_Vehicle_Count'] / pkdt['Parking_Spaces'], 1)
pkdt['Occupancy'] = Occupancy
pkdt = pkdt.dropna()
pkdt = pkdt.drop('Total_Vehicle_Count', axis = 1)

In [6]:
#retrivev 'hour' column from general datetime
pkdt['Date Time'] = pd.to_datetime(pkdt['Date Time'], format = '%m-%d-%y %H:%M:%S')
times = pd.DatetimeIndex(pkdt['Date Time'])
pkdt['Hour'] = times.hour
#for 'Hour' column, replace 0 with 24
pkdt = pkdt.replace({'Hour': 0}, 24)

In [7]:
#group data by 'hour' and 'Unidesc' and average other columns
pkdt = pkdt.groupby(['Hour', 'Unitdesc'], as_index = False)['Parking_Spaces', 'Occupancy'].mean()

In [8]:
#group data by 'hour' and 'Unidesc' and average other columns
pkdt = pkdt.groupby(['Hour', 'Unitdesc'], as_index = False)['Parking_Spaces', 'Occupancy'].mean()

In [9]:
# Rename the column names to all_capital
pkdt = pkdt.rename(index=str, columns={"Hour": "HOUR", "Unitdesc": "UNITDESC",
                                'Parking_Spaces':"PARKING_SPACE", 
                                'Occupancy': "OCCUPANCY"})
pkdt.to_csv('occupancy_per_hour.csv')

In [10]:
# After cleaning the Blockface, some streets in APS are not included
# in BF. Drop the extra streets.
bfdf = pd.read_csv('Rate_Limit.csv')
aps = cln.subset([pkdt, 'UNITDESC'], [bfdf, 'UNITDESC'])

In [11]:
# Check the number of streets
print(len(bfdf.UNITDESC.unique()))
print(len(aps.UNITDESC.unique()))

1234
1234


In [12]:
aps.to_csv('occupancy_per_hour.csv')
